In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [20]:

api_key = os.environ["GOOGLE_API_KEY"]


In [ ]:
# Etapa 1 - ETL

# SUBIR OS PDFs
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("documentos_curso/")
docs = loader.load()

# Gerar os chunks e fazer o embeding


# Subi para o banco de dados chromaDb


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " ", ""]
)

chunks = text_splitter.split_documents(docs)

for i, chunk in enumerate(chunks, start=1):
    print(f"Chunk {i} – origem: {chunk.metadata.get('source')}")
    print(chunk.page_content)
    print("-" * 80)


In [25]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", api_key= api_key)
embeddings_model

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001EFDA3078A0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [27]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_model)

print("Banco de dados criado com sucesso!")

Banco de dados criado com sucesso!


In [28]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# 1. Recuperador Lexical (BM25)
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 5

# 2. Recuperador Vetorial (a partir do nosso ChromaDB)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


# 3. EnsembleRetriever para combinar os resultados
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.4, 0.6]
)

print("Recuperador de Busca Hibrida Configurado")

Recuperador de Busca Hibrida Configurado


In [40]:
from langchain.chains import ConversationalRetrievalChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory

llm = ChatGoogleGenerativeAI(model="models/gemini-2.0-flash", temperature=0, api_key=api_key)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    output_key="answer",
    return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=ensemble_retriever,
    memory=memory,
    verbose=False,
    return_source_documents=True
)

In [41]:
resposta1 = qa_chain.invoke({"question": "O que é chunking adaptativo?"})
print(resposta1['answer'])

Chunking adaptativo é a técnica de dividir documentos longos em pedaços (chunks) de forma inteligente, preservando o contexto semântico. A estratégia de chunking deve ser adaptada ao tipo de dado, como textos contínuos, tabelas, código-fonte ou documentos estruturados, que requerem abordagens diferentes. As abordagens incluem: Fixed-Size Chunking (divisão por número fixo de tokens ou caracteres), Sliding Window (sobreposição para manter contexto entre chunks) e Recursive Splitting (divisão baseada na estrutura semântica do texto).


In [42]:
resposta2 = qa_chain.invoke({"question": "E quais as principais estratégias?"})
print(resposta2['answer'])

As principais estratégias de chunking adaptativo são:

*   **Fixed-Size Chunking:** Divisão por número fixo de tokens ou caracteres.
*   **Sliding Window:** Sobreposição (overlap) para manter o contexto entre os chunks.
*   **Recursive Splitting:** Divisão baseada na estrutura semântica do texto.


In [45]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

# 1. Crie um conjunto de dados para avaliação
perguntas = [
    "O que é RAG e qual problema ele soluciona?",
    "Quais os componentes essenciais do RAG?",
    "Qual a diferença entre busca lexical e semântica?",
    "O que mede a métrica faithfulness do RAGAS?"
]
respostas_ouro = [
    "RAG (Retrieval-Augmented Generation) é uma arquitetura que combina um motor de busca para recuperar informações com um LLM para gerar respostas. Ele soluciona problemas como alucinações e conhecimento desatualizado dos LLMs.",
    "Os componentes essenciais são: Embeddings, Banco de Dados Vetorial, Chunking e um Modelo de Linguagem (LLM).",
    "Busca lexical (como BM25) encontra correspondências exatas de termos, enquanto a busca semântica captura o significado e o contexto, mesmo com palavras diferentes.",
    "A métrica Faithfulness mede se a resposta gerada é suportada e factualmente consistente com os documentos recuperados, evitando alucinações."
]

# 2. Gere as respostas e contextos com a nossa cadeia
respostas_geradas = []
contextos_recuperados = []
for question in perguntas:
    result = qa_chain.invoke({"question": question})
    respostas_geradas.append(result['answer'])
    contextos_recuperados.append([doc.page_content for doc in result['source_documents']])

# 3. Crie o dataset no formato esperado pelo RAGAS
dataset_dict = {
    'question': perguntas,
    'answer': respostas_geradas,
    'contexts': contextos_recuperados,
    'ground_truth': respostas_ouro
}
dataset = Dataset.from_dict(dataset_dict)

# 4. Execute a avaliação, agora usando os modelos do Google
evaluation_result = evaluate(
    dataset=dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
    llm=ChatGoogleGenerativeAI(model="models/gemini-2.0-flash"),
    embeddings=embeddings_model
)

# 5. Analise os resultados
df_resultados = evaluation_result.to_pandas()
print("\nResultados da Avaliação com RAGAS:")
display(df_resultados)

Evaluating:   6%|▋         | 1/16 [00:02<00:39,  2.67s/it]Exception raised in Job[13]: IndexError(list index out of range)
Exception raised in Job[5]: IndexError(list index out of range)
Exception raised in Job[1]: IndexError(list index out of range)
Evaluating: 100%|██████████| 16/16 [02:13<00:00,  8.31s/it]



Resultados da Avaliação com RAGAS:


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,O que é RAG e qual problema ele soluciona?,[POR QUE RAG É A REVOLUÇÃO\nReduz Alucinações\...,RAG (Retrieval-Augmented Generation) é uma téc...,RAG (Retrieval-Augmented Generation) é uma arq...,0.642857,NaN,0.591667,1.0
1,Quais os componentes essenciais do RAG?,[POR QUE RAG É A REVOLUÇÃO\nReduz Alucinações\...,Os componentes essenciais do RAG são:\n* Embed...,"Os componentes essenciais são: Embeddings, Ban...",1.000000,NaN,0.500000,1.0
2,Qual a diferença entre busca lexical e semântica?,[alura\nHybrid Search (Busca Híbrida)\nBusca s...,A busca semântica captura o significado e cont...,Busca lexical (como BM25) encontra correspondê...,1.000000,NaN,0.916667,1.0
3,O que mede a métrica faithfulness do RAGAS?,[alura\nMétricas de Geração\nO RAGAS fornece m...,A métrica faithfulness do RAGAS mede se a resp...,A métrica Faithfulness mede se a resposta gera...,1.000000,NaN,0.750000,1.0
